In [17]:
from sage.all import *
import random

def degree_configuration(graph):
    degree_counts = [0, 0, 0, 0]
    for degree in graph.degree():
        degree_counts[degree - 1] += 1
    return tuple(degree_counts)

def generate_initial_graph(n):
    """
    Funkcija generira drevo na n vozliščih z omejtivijo max(g.degree()) <= 4.
    Args:
        n (int): Število vozlišč.

    Returns:
        Graph: Drevo.
    """
    g = Graph()  #prazen graf
    g.add_vertices(range(n))  #dodamo n vozlišč
    available = list(range(n))  #ustvarimo seznam še nepovezanih vozlišč
    connected = [available.pop()]  #in seznam povezanih vozlišč

    while available: #dokler niso vsa povezana
        u = random.choice(connected)  #izberemo naključno vozlišče, ki je že povezano
        v = available.pop()  #in vozlišče, ki še ni
        if g.degree(u) < 4: 
            g.add_edge(u, v)  
            connected.append(v)  #če ne krši omejitev ju povežemo
        else:
            available.append(v) 

    return g

def mutate_graph(g, u, v):
    """
    Funkcija doda oziroma odstrani povezavo med izbranima vozliščema u in v.

    Args:
    Graph: Graf g.
    Vertice: Vozlišče u.
    Vertice: Vozlišče v.

    Returns:
    Graph: Graf g.
    """
    if g.has_edge(u, v):
        g.delete_edge(u,v)
        if g.is_connected():
            return g
        else:
            g.add_edge(u,v) #če krši omejitve vrnemo povezavo
            return g
    else:  
        g.add_edge(u, v)
        if max(g.degree()) <= 4:
            return g
        else:
            g.delete_edge(u, v)
            return g


def hill_climbing(n, f, iterations):
    """
    Opis algoritma v poročilu.

    Args:
        n (int): Število vozlišč.
        f (function): Function of n, e.g., lambda n: 1/n or lambda n: c.
        iterations (int): Število iteracij.

    Returns:
        Graph: Graf z najvišjim sigma totalnim indeksom.
        float: Sigma totalni indeks.
    """
    f_n = f(n)
    current_graph = generate_initial_graph(n)
    degree_counts = current_graph.degree().copy()
    
    
    
    
    for _ in range(iterations):
        vertices = current_graph.vertices()
        u, v = random.sample(vertices, 2)

    #izberemo dve naključni vozlišči
    #namesto, da vsakič znova računamo sigma totalni indeks, preverimo če bo mutacija povečala prispevek, kot ju te dve vozlišči data pred njo   
        original_contribution = 0
        for w in vertices:
            if w != u:
                original_contribution += abs(degree_counts[u] - degree_counts[w]) ** f_n
            if w != v and w != u: #torej use kjer nastopa w in u, pa w in v, pa enkrat u in v
                original_contribution += abs(degree_counts[v] - degree_counts[w]) ** f_n

        

        new_graph = mutate_graph(current_graph.copy(), u, v) 
        new_degree_counts = new_graph.degree() 
        


        
        new_contribution = 0
        for w in vertices:
            if w != u:
                new_contribution += abs(new_degree_counts[u] - new_degree_counts[w]) ** f_n
            if w != v and w!= u:
                new_contribution += abs(new_degree_counts[v] - new_degree_counts[w]) ** f_n


        #Če je prispevek boljši, mutacijo grafa obdržimo
        if new_contribution > original_contribution:
            current_graph = new_graph
            degree_counts = new_degree_counts

    
    final_score = 0
    for x, y in Subsets(vertices, 2):
        final_score += (abs(current_graph.degree(x) - current_graph.degree(y)) ** f_n )
    


    return current_graph, final_score



<>:57: DeprecationWarning: invalid escape sequence '\s'
<>:57: DeprecationWarning: invalid escape sequence '\s'
/tmp/ipykernel_1035/1025950501.py:57: DeprecationWarning: invalid escape sequence '\s'
  """


In [18]:
from sage.all import *
import random
import math

def degree_configuration(graph):
    degree_counts = [0, 0, 0, 0]
    for degree in graph.degree():
        degree_counts[degree - 1] += 1
    return tuple(degree_counts)

def generate_initial_graph(n):
    g = Graph()
    g.add_vertices(range(n))
    available = list(range(n))
    connected = [available.pop()]

    while available:
        u = random.choice(connected)
        v = available.pop()
        if g.degree(u) < 4:
            g.add_edge(u, v)
            connected.append(v)
        else:
            available.append(v)

    return g

def mutate_graph(g, u, v):

    if g.has_edge(u, v):
        g.delete_edge(u,v)
        if g.is_connected():
            return g
        else:
            g.add_edge(u,v)
            return g
    else:  
        g.add_edge(u, v)
        if max(g.degree()) <= 4:
            return g
        else:
            g.delete_edge(u, v)
            return g

def hill_climbing_sa(n, f, iterations, T=1.0, alpha=0.99):
    """
    Opis algoritma v poročilu.

    Args:
        n (int): število vozlišč
        f (function): f(n)
        iterations (int): število iteracij
        T (float): začetna temperatura
        alpha (float): stopnja ohlajanja

    Returns:
        Graph: Graf z najvišjim sigma totalnim indeksom.
        float: Sigma totalni indeks.
    """
    f_n = f(n)
    current_graph = generate_initial_graph(n)
    degree_counts = current_graph.degree().copy()
    
    

    for _ in range(iterations):
        vertices = current_graph.vertices()
        u, v = random.sample(vertices, 2)

        
        original_contribution = 0
        for w in vertices:
            if w != u:
                original_contribution += abs(degree_counts[u] - degree_counts[w]) ** f_n
            if w != v and w != u:
                original_contribution += abs(degree_counts[v] - degree_counts[w]) ** f_n

        new_graph = mutate_graph(current_graph.copy(), u, v) 
        new_degree_counts = new_graph.degree() 

        
        new_contribution = 0
        for w in vertices:
            if w != u:
                new_contribution += abs(new_degree_counts[u] - new_degree_counts[w]) ** f_n
            if w != v and w!= u:
                new_contribution += abs(new_degree_counts[v] - new_degree_counts[w]) ** f_n

        
        delta = new_contribution - original_contribution

        if delta > 0 or random.random() < math.exp(delta / T):
            current_graph = new_graph
            degree_counts = new_degree_counts

        
        
    
        T *= alpha  #ohlajamo temperaturo

    final_score = 0
    for x, y in Subsets(vertices, 2):
        final_score += (abs(current_graph.degree(x) - current_graph.degree(y)) ** f_n )

    return current_graph, final_score




In [3]:
print("Hill climbing algoritm:")
iterations = 100000
n = 15
f = lambda n: 1/n


zmagovalec, sigma = hill_climbing(n, f, iterations)
print(f"Najden maksimum na {n} vozliščih za f(n) = 1/n = {round(1/n,2)}: {degree_configuration(zmagovalec)}, Sigma = {round(sigma,2)}")

c = 0
while c < 1:
    f = lambda n: c
    zmagovalec, sigma = hill_climbing(n, f, iterations)
    print(f"Najden maksimum na {n} vozliščih za f(n) = {c}: {degree_configuration(zmagovalec)}, Sigma = {round(sigma,2)}")
    c += 0.1
    c = round(c,2)

print("Simulated annealing algoritm:")
f = lambda n: 1/n
T_initial = 1.0
alpha = 0.99

zmagovalec, sigma = hill_climbing_sa(n, f, iterations, T_initial, alpha)
print(f"Najden maksimum na {n} vozliščih za f(n) = 1/n = {round(1/n,2)}: {degree_configuration(zmagovalec)}, Sigma = {round(sigma,2)}")

c = 0
while c < 1:
    f = lambda n: c
    zmagovalec, sigma = hill_climbing_sa(n, f, iterations, T_initial, alpha)
    print(f"Najden maksimum na {n} vozliščih za f(n) = {c}: {degree_configuration(zmagovalec)}, Sigma = {round(sigma,2)}")
    c += 0.1
    c = round(c,2)

Hill climbing algoritm:
Najden maksimum na 15 vozliščih za f(n) = 1/n = 0.07: (4, 3, 4, 4), Sigma = 86.54
Najden maksimum na 15 vozliščih za f(n) = 0: (6, 7, 0, 2), Sigma = 105.0
Najden maksimum na 15 vozliščih za f(n) = 0.1: (4, 3, 4, 4), Sigma = 87.87
Najden maksimum na 15 vozliščih za f(n) = 0.2: (4, 3, 4, 4), Sigma = 92.1
Najden maksimum na 15 vozliščih za f(n) = 0.3: (5, 3, 3, 4), Sigma = 97.05
Najden maksimum na 15 vozliščih za f(n) = 0.4: (5, 2, 3, 5), Sigma = 102.78
Najden maksimum na 15 vozliščih za f(n) = 0.5: (5, 2, 3, 5), Sigma = 109.66
Najden maksimum na 15 vozliščih za f(n) = 0.6: (5, 2, 3, 5), Sigma = 117.22
Najden maksimum na 15 vozliščih za f(n) = 0.7: (5, 2, 3, 5), Sigma = 125.55
Najden maksimum na 15 vozliščih za f(n) = 0.8: (6, 1, 2, 6), Sigma = 138.04
Najden maksimum na 15 vozliščih za f(n) = 0.9: (6, 1, 2, 6), Sigma = 150.35
Simulated annealing algoritm:
Najden maksimum na 15 vozliščih za f(n) = 1/n = 0.07: (4, 3, 4, 4), Sigma = 86.54
Najden maksimum na 15 vozlišč

In [4]:
print("Hill climbing algoritm:")
iterations = 100000
n = 16
f = lambda n: 1/n


zmagovalec, sigma = hill_climbing(n, f, iterations)
print(f"Najden maksimum na {n} vozliščih za f(n) = 1/n = {round(1/n,2)}: {degree_configuration(zmagovalec)}, Sigma = {round(sigma,2)}")

c = 0
while c < 1:
    f = lambda n: c
    zmagovalec, sigma = hill_climbing(n, f, iterations)
    print(f"Najden maksimum na {n} vozliščih za f(n) = {c}: {degree_configuration(zmagovalec)}, Sigma = {round(sigma,2)}")
    c += 0.1
    c = round(c,2)

print("Simulated annealing algoritm:")
f = lambda n: 1/n
T_initial = 1.0
alpha = 0.99

zmagovalec, sigma = hill_climbing_sa(n, f, iterations, T_initial, alpha)
print(f"Najden maksimum na {n} vozliščih za f(n) = 1/n = {round(1/n,2)}: {degree_configuration(zmagovalec)}, Sigma = {round(sigma,2)}")

c = 0
while c < 1:
    f = lambda n: c
    zmagovalec, sigma = hill_climbing_sa(n, f, iterations, T_initial, alpha)
    print(f"Najden maksimum na {n} vozliščih za f(n) = {c}: {degree_configuration(zmagovalec)}, Sigma = {round(sigma,2)}")
    c += 0.1
    c = round(c,2)

Hill climbing algoritm:
Najden maksimum na 16 vozliščih za f(n) = 1/n = 0.06: (4, 4, 4, 4), Sigma = 98.55
Najden maksimum na 16 vozliščih za f(n) = 0: (9, 2, 3, 2), Sigma = 120.0
Najden maksimum na 16 vozliščih za f(n) = 0.1: (4, 4, 4, 4), Sigma = 100.15
Najden maksimum na 16 vozliščih za f(n) = 0.2: (4, 4, 4, 4), Sigma = 104.69
Najden maksimum na 16 vozliščih za f(n) = 0.3: (5, 3, 3, 5), Sigma = 110.69
Najden maksimum na 16 vozliščih za f(n) = 0.4: (5, 3, 3, 5), Sigma = 117.38
Najden maksimum na 16 vozliščih za f(n) = 0.5: (5, 3, 3, 5), Sigma = 124.73
Najden maksimum na 16 vozliščih za f(n) = 0.6: (6, 2, 2, 6), Sigma = 133.97
Najden maksimum na 16 vozliščih za f(n) = 0.7: (6, 2, 2, 6), Sigma = 144.66
Najden maksimum na 16 vozliščih za f(n) = 0.8: (6, 2, 2, 6), Sigma = 156.48
Najden maksimum na 16 vozliščih za f(n) = 0.9: (6, 1, 2, 7), Sigma = 170.35
Simulated annealing algoritm:
Najden maksimum na 16 vozliščih za f(n) = 1/n = 0.06: (4, 4, 4, 4), Sigma = 98.55
Najden maksimum na 16 voz

In [5]:
print("Hill climbing algoritm:")
iterations = 100000
n = 17
f = lambda n: 1/n


zmagovalec, sigma = hill_climbing(n, f, iterations)
print(f"Najden maksimum na {n} vozliščih za f(n) = 1/n = {round(1/n,2)}: {degree_configuration(zmagovalec)}, Sigma = {round(sigma,2)}")

c = 0
while c < 1:
    f = lambda n: c
    zmagovalec, sigma = hill_climbing(n, f, iterations)
    print(f"Najden maksimum na {n} vozliščih za f(n) = {c}: {degree_configuration(zmagovalec)}, Sigma = {round(sigma,2)}")
    c += 0.1
    c = round(c,2)

print("Simulated annealing algoritm:")
f = lambda n: 1/n
T_initial = 1.0
alpha = 0.99

zmagovalec, sigma = hill_climbing_sa(n, f, iterations, T_initial, alpha)
print(f"Najden maksimum na {n} vozliščih za f(n) = 1/n = {round(1/n,2)}: {degree_configuration(zmagovalec)}, Sigma = {round(sigma,2)}")

c = 0
while c < 1:
    f = lambda n: c
    zmagovalec, sigma = hill_climbing_sa(n, f, iterations, T_initial, alpha)
    print(f"Najden maksimum na {n} vozliščih za f(n) = {c}: {degree_configuration(zmagovalec)}, Sigma = {round(sigma,2)}")
    c += 0.1
    c = round(c,2)

Hill climbing algoritm:
Najden maksimum na 17 vozliščih za f(n) = 1/n = 0.06: (4, 4, 4, 5), Sigma = 110.83
Najden maksimum na 17 vozliščih za f(n) = 0: (8, 3, 6, 0), Sigma = 136.0
Najden maksimum na 17 vozliščih za f(n) = 0.1: (4, 4, 4, 5), Sigma = 112.91
Najden maksimum na 17 vozliščih za f(n) = 0.2: (5, 4, 3, 5), Sigma = 118.35
Najden maksimum na 17 vozliščih za f(n) = 0.3: (5, 4, 3, 5), Sigma = 124.85
Najden maksimum na 17 vozliščih za f(n) = 0.4: (5, 3, 3, 6), Sigma = 132.1
Najden maksimum na 17 vozliščih za f(n) = 0.5: (6, 3, 2, 6), Sigma = 140.78
Najden maksimum na 17 vozliščih za f(n) = 0.6: (6, 3, 2, 6), Sigma = 151.07
Najden maksimum na 17 vozliščih za f(n) = 0.7: (5, 3, 3, 6), Sigma = 160.34
Najden maksimum na 17 vozliščih za f(n) = 0.8: (6, 2, 2, 7), Sigma = 176.41
Najden maksimum na 17 vozliščih za f(n) = 0.9: (6, 2, 2, 7), Sigma = 191.41
Simulated annealing algoritm:
Najden maksimum na 17 vozliščih za f(n) = 1/n = 0.06: (4, 4, 4, 5), Sigma = 110.83
Najden maksimum na 17 vo

In [6]:
print("Hill climbing algoritm:")
iterations = 100000
n = 18
f = lambda n: 1/n


zmagovalec, sigma = hill_climbing(n, f, iterations)
print(f"Najden maksimum na {n} vozliščih za f(n) = 1/n = {round(1/n,2)}: {degree_configuration(zmagovalec)}, Sigma = {round(sigma,2)}")

c = 0
while c < 1:
    f = lambda n: c
    zmagovalec, sigma = hill_climbing(n, f, iterations)
    print(f"Najden maksimum na {n} vozliščih za f(n) = {c}: {degree_configuration(zmagovalec)}, Sigma = {round(sigma,2)}")
    c += 0.1
    c = round(c,2)

print("Simulated annealing algoritm:")
f = lambda n: 1/n
T_initial = 1.0
alpha = 0.99

zmagovalec, sigma = hill_climbing_sa(n, f, iterations, T_initial, alpha)
print(f"Najden maksimum na {n} vozliščih za f(n) = 1/n = {round(1/n,2)}: {degree_configuration(zmagovalec)}, Sigma = {round(sigma,2)}")

c = 0
while c < 1:
    f = lambda n: c
    zmagovalec, sigma = hill_climbing_sa(n, f, iterations, T_initial, alpha)
    print(f"Najden maksimum na {n} vozliščih za f(n) = {c}: {degree_configuration(zmagovalec)}, Sigma = {round(sigma,2)}")
    c += 0.1
    c = round(c,2)

Hill climbing algoritm:
Najden maksimum na 18 vozliščih za f(n) = 1/n = 0.06: (4, 5, 4, 5), Sigma = 123.87
Najden maksimum na 18 vozliščih za f(n) = 0: (8, 6, 2, 2), Sigma = 153.0
Najden maksimum na 18 vozliščih za f(n) = 0.1: (5, 4, 5, 4), Sigma = 126.27
Najden maksimum na 18 vozliščih za f(n) = 0.2: (5, 4, 3, 6), Sigma = 132.17
Najden maksimum na 18 vozliščih za f(n) = 0.3: (6, 3, 4, 5), Sigma = 139.73
Najden maksimum na 18 vozliščih za f(n) = 0.4: (5, 4, 3, 6), Sigma = 148.02
Najden maksimum na 18 vozliščih za f(n) = 0.5: (7, 2, 3, 6), Sigma = 157.42
Najden maksimum na 18 vozliščih za f(n) = 0.6: (6, 2, 4, 6), Sigma = 168.16
Najden maksimum na 18 vozliščih za f(n) = 0.7: (6, 3, 2, 7), Sigma = 182.23
Najden maksimum na 18 vozliščih za f(n) = 0.8: (8, 1, 2, 7), Sigma = 198.91
Najden maksimum na 18 vozliščih za f(n) = 0.9: (7, 2, 1, 8), Sigma = 217.44
Simulated annealing algoritm:
Najden maksimum na 18 vozliščih za f(n) = 1/n = 0.06: (4, 5, 4, 5), Sigma = 123.87
Najden maksimum na 18 v

In [7]:
print("Hill climbing algoritm:")
iterations = 100000
n = 19
f = lambda n: 1/n


zmagovalec, sigma = hill_climbing(n, f, iterations)
print(f"Najden maksimum na {n} vozliščih za f(n) = 1/n = {round(1/n,2)}: {degree_configuration(zmagovalec)}, Sigma = {round(sigma,2)}")

c = 0
while c < 1:
    f = lambda n: c
    zmagovalec, sigma = hill_climbing(n, f, iterations)
    print(f"Najden maksimum na {n} vozliščih za f(n) = {c}: {degree_configuration(zmagovalec)}, Sigma = {round(sigma,2)}")
    c += 0.1
    c = round(c,2)

print("Simulated annealing algoritm:")
f = lambda n: 1/n
T_initial = 1.0
alpha = 0.99

zmagovalec, sigma = hill_climbing_sa(n, f, iterations, T_initial, alpha)
print(f"Najden maksimum na {n} vozliščih za f(n) = 1/n = {round(1/n,2)}: {degree_configuration(zmagovalec)}, Sigma = {round(sigma,2)}")

c = 0
while c < 1:
    f = lambda n: c
    zmagovalec, sigma = hill_climbing_sa(n, f, iterations, T_initial, alpha)
    print(f"Najden maksimum na {n} vozliščih za f(n) = {c}: {degree_configuration(zmagovalec)}, Sigma = {round(sigma,2)}")
    c += 0.1
    c = round(c,2)

Hill climbing algoritm:
Najden maksimum na 19 vozliščih za f(n) = 1/n = 0.05: (5, 4, 5, 5), Sigma = 138.16
Najden maksimum na 19 vozliščih za f(n) = 0: (11, 2, 3, 3), Sigma = 171.0
Najden maksimum na 19 vozliščih za f(n) = 0.1: (5, 4, 5, 5), Sigma = 141.13
Najden maksimum na 19 vozliščih za f(n) = 0.2: (6, 4, 4, 5), Sigma = 147.91
Najden maksimum na 19 vozliščih za f(n) = 0.3: (6, 4, 4, 5), Sigma = 155.88
Najden maksimum na 19 vozliščih za f(n) = 0.4: (6, 3, 4, 6), Sigma = 165.29
Najden maksimum na 19 vozliščih za f(n) = 0.5: (7, 3, 3, 6), Sigma = 175.9
Najden maksimum na 19 vozliščih za f(n) = 0.6: (7, 2, 3, 7), Sigma = 188.78
Najden maksimum na 19 vozliščih za f(n) = 0.7: (7, 2, 3, 7), Sigma = 203.58
Najden maksimum na 19 vozliščih za f(n) = 0.8: (8, 1, 2, 8), Sigma = 221.91
Najden maksimum na 19 vozliščih za f(n) = 0.9: (7, 1, 3, 8), Sigma = 238.64
Simulated annealing algoritm:
Najden maksimum na 19 vozliščih za f(n) = 1/n = 0.05: (5, 4, 5, 5), Sigma = 138.16
Najden maksimum na 19 v

In [8]:
print("Hill climbing algoritm:")
iterations = 100000
n = 20
f = lambda n: 1/n


zmagovalec, sigma = hill_climbing(n, f, iterations)
print(f"Najden maksimum na {n} vozliščih za f(n) = 1/n = {round(1/n,2)}: {degree_configuration(zmagovalec)}, Sigma = {round(sigma,2)}")

c = 0
while c < 1:
    f = lambda n: c
    zmagovalec, sigma = hill_climbing(n, f, iterations)
    print(f"Najden maksimum na {n} vozliščih za f(n) = {c}: {degree_configuration(zmagovalec)}, Sigma = {round(sigma,2)}")
    c += 0.1
    c = round(c,2)

print("Simulated annealing algoritm:")
f = lambda n: 1/n
T_initial = 1.0
alpha = 0.99

zmagovalec, sigma = hill_climbing_sa(n, f, iterations, T_initial, alpha)
print(f"Najden maksimum na {n} vozliščih za f(n) = 1/n = {round(1/n,2)}: {degree_configuration(zmagovalec)}, Sigma = {round(sigma,2)}")

c = 0
while c < 1:
    f = lambda n: c
    zmagovalec, sigma = hill_climbing_sa(n, f, iterations, T_initial, alpha)
    print(f"Najden maksimum na {n} vozliščih za f(n) = {c}: {degree_configuration(zmagovalec)}, Sigma = {round(sigma,2)}")
    c += 0.1
    c = round(c,2)

Hill climbing algoritm:
Najden maksimum na 20 vozliščih za f(n) = 1/n = 0.05: (5, 5, 5, 5), Sigma = 153.17
Najden maksimum na 20 vozliščih za f(n) = 0: (10, 5, 2, 3), Sigma = 190.0
Najden maksimum na 20 vozliščih za f(n) = 0.1: (5, 5, 5, 5), Sigma = 156.49
Najden maksimum na 20 vozliščih za f(n) = 0.2: (6, 4, 4, 6), Sigma = 163.98
Najden maksimum na 20 vozliščih za f(n) = 0.3: (6, 4, 4, 6), Sigma = 173.15
Najden maksimum na 20 vozliščih za f(n) = 0.4: (6, 4, 4, 6), Sigma = 183.2
Najden maksimum na 20 vozliščih za f(n) = 0.5: (7, 3, 3, 7), Sigma = 195.27
Najden maksimum na 20 vozliščih za f(n) = 0.6: (7, 3, 3, 7), Sigma = 209.39
Najden maksimum na 20 vozliščih za f(n) = 0.7: (8, 2, 2, 8), Sigma = 226.07
Najden maksimum na 20 vozliščih za f(n) = 0.8: (8, 2, 2, 8), Sigma = 245.84
Najden maksimum na 20 vozliščih za f(n) = 0.9: (7, 3, 1, 9), Sigma = 265.78
Simulated annealing algoritm:
Najden maksimum na 20 vozliščih za f(n) = 1/n = 0.05: (5, 5, 5, 5), Sigma = 153.17
Najden maksimum na 20 v

In [9]:
print("Hill climbing algoritm:")
iterations = 100000
n = 50
f = lambda n: 1/n


zmagovalec, sigma = hill_climbing(n, f, iterations)
print(f"Najden maksimum na {n} vozliščih za f(n) = 1/n = {round(1/n,2)}: {degree_configuration(zmagovalec)}, Sigma = {round(sigma,2)}")

c = 0
while c < 1:
    f = lambda n: c
    zmagovalec, sigma = hill_climbing(n, f, iterations)
    print(f"Najden maksimum na {n} vozliščih za f(n) = {c}: {degree_configuration(zmagovalec)}, Sigma = {round(sigma,2)}")
    c += 0.1
    c = round(c,2)

print("Simulated annealing algoritm:")
f = lambda n: 1/n
T_initial = 1.0
alpha = 0.99

zmagovalec, sigma = hill_climbing_sa(n, f, iterations, T_initial, alpha)
print(f"Najden maksimum na {n} vozliščih za f(n) = 1/n = {round(1/n,2)}: {degree_configuration(zmagovalec)}, Sigma = {round(sigma,2)}")

c = 0
while c < 1:
    f = lambda n: c
    zmagovalec, sigma = hill_climbing_sa(n, f, iterations, T_initial, alpha)
    print(f"Najden maksimum na {n} vozliščih za f(n) = {c}: {degree_configuration(zmagovalec)}, Sigma = {round(sigma,2)}")
    c += 0.1
    c = round(c,2)

Hill climbing algoritm:
Najden maksimum na 50 vozliščih za f(n) = 1/n = 0.02: (12, 13, 12, 13), Sigma = 944.83
Najden maksimum na 50 vozliščih za f(n) = 0: (23, 12, 9, 6), Sigma = 1225.0
Najden maksimum na 50 vozliščih za f(n) = 0.1: (13, 12, 11, 14), Sigma = 978.46
Najden maksimum na 50 vozliščih za f(n) = 0.2: (13, 12, 11, 14), Sigma = 1025.97
Najden maksimum na 50 vozliščih za f(n) = 0.3: (14, 11, 10, 15), Sigma = 1081.48
Najden maksimum na 50 vozliščih za f(n) = 0.4: (15, 10, 9, 16), Sigma = 1145.7
Najden maksimum na 50 vozliščih za f(n) = 0.5: (17, 8, 9, 16), Sigma = 1220.51
Najden maksimum na 50 vozliščih za f(n) = 0.6: (19, 6, 7, 18), Sigma = 1308.44
Najden maksimum na 50 vozliščih za f(n) = 0.7: (20, 5, 6, 19), Sigma = 1413.18
Najden maksimum na 50 vozliščih za f(n) = 0.8: (21, 4, 5, 20), Sigma = 1537.56
Najden maksimum na 50 vozliščih za f(n) = 0.9: (21, 4, 3, 22), Sigma = 1685.57
Simulated annealing algoritm:
Najden maksimum na 50 vozliščih za f(n) = 1/n = 0.02: (12, 13, 12, 

In [10]:
print("Hill climbing algoritm:")
iterations = 100000
n = 51
f = lambda n: 1/n


zmagovalec, sigma = hill_climbing(n, f, iterations)
print(f"Najden maksimum na {n} vozliščih za f(n) = 1/n = {round(1/n,2)}: {degree_configuration(zmagovalec)}, Sigma = {round(sigma,2)}")

c = 0
while c < 1:
    f = lambda n: c
    zmagovalec, sigma = hill_climbing(n, f, iterations)
    print(f"Najden maksimum na {n} vozliščih za f(n) = {c}: {degree_configuration(zmagovalec)}, Sigma = {round(sigma,2)}")
    c += 0.1
    c = round(c,2)

print("Simulated annealing algoritm:")
f = lambda n: 1/n
T_initial = 1.0
alpha = 0.99

zmagovalec, sigma = hill_climbing_sa(n, f, iterations, T_initial, alpha)
print(f"Najden maksimum na {n} vozliščih za f(n) = 1/n = {round(1/n,2)}: {degree_configuration(zmagovalec)}, Sigma = {round(sigma,2)}")

c = 0
while c < 1:
    f = lambda n: c
    zmagovalec, sigma = hill_climbing_sa(n, f, iterations, T_initial, alpha)
    print(f"Najden maksimum na {n} vozliščih za f(n) = {c}: {degree_configuration(zmagovalec)}, Sigma = {round(sigma,2)}")
    c += 0.1
    c = round(c,2)

Hill climbing algoritm:
Najden maksimum na 51 vozliščih za f(n) = 1/n = 0.02: (13, 12, 13, 13), Sigma = 983.13
Najden maksimum na 51 vozliščih za f(n) = 0: (23, 14, 7, 7), Sigma = 1275.0
Najden maksimum na 51 vozliščih za f(n) = 0.1: (14, 12, 12, 13), Sigma = 1018.39
Najden maksimum na 51 vozliščih za f(n) = 0.2: (14, 11, 12, 14), Sigma = 1068.04
Najden maksimum na 51 vozliščih za f(n) = 0.3: (15, 10, 11, 15), Sigma = 1125.65
Najden maksimum na 51 vozliščih za f(n) = 0.4: (16, 9, 10, 16), Sigma = 1192.4
Najden maksimum na 51 vozliščih za f(n) = 0.5: (17, 8, 9, 17), Sigma = 1270.27
Najden maksimum na 51 vozliščih za f(n) = 0.6: (18, 7, 8, 18), Sigma = 1361.59
Najden maksimum na 51 vozliščih za f(n) = 0.7: (20, 5, 6, 20), Sigma = 1470.46
Najden maksimum na 51 vozliščih za f(n) = 0.8: (21, 4, 5, 21), Sigma = 1600.1
Najden maksimum na 51 vozliščih za f(n) = 0.9: (21, 4, 3, 23), Sigma = 1752.48
Simulated annealing algoritm:
Najden maksimum na 51 vozliščih za f(n) = 1/n = 0.02: (13, 12, 13, 

In [11]:
print("Hill climbing algoritm:")
iterations = 100000
n = 52
f = lambda n: 1/n


zmagovalec, sigma = hill_climbing(n, f, iterations)
print(f"Najden maksimum na {n} vozliščih za f(n) = 1/n = {round(1/n,2)}: {degree_configuration(zmagovalec)}, Sigma = {round(sigma,2)}")

c = 0
while c < 1:
    f = lambda n: c
    zmagovalec, sigma = hill_climbing(n, f, iterations)
    print(f"Najden maksimum na {n} vozliščih za f(n) = {c}: {degree_configuration(zmagovalec)}, Sigma = {round(sigma,2)}")
    c += 0.1
    c = round(c,2)

print("Simulated annealing algoritm:")
f = lambda n: 1/n
T_initial = 1.0
alpha = 0.99

zmagovalec, sigma = hill_climbing_sa(n, f, iterations, T_initial, alpha)
print(f"Najden maksimum na {n} vozliščih za f(n) = 1/n = {round(1/n,2)}: {degree_configuration(zmagovalec)}, Sigma = {round(sigma,2)}")

c = 0
while c < 1:
    f = lambda n: c
    zmagovalec, sigma = hill_climbing_sa(n, f, iterations, T_initial, alpha)
    print(f"Najden maksimum na {n} vozliščih za f(n) = {c}: {degree_configuration(zmagovalec)}, Sigma = {round(sigma,2)}")
    c += 0.1
    c = round(c,2)

Hill climbing algoritm:
Najden maksimum na 52 vozliščih za f(n) = 1/n = 0.02: (13, 13, 13, 13), Sigma = 1022.14
Najden maksimum na 52 vozliščih za f(n) = 0: (22, 17, 6, 7), Sigma = 1326.0
Najden maksimum na 52 vozliščih za f(n) = 0.1: (14, 12, 12, 14), Sigma = 1058.88
Najden maksimum na 52 vozliščih za f(n) = 0.2: (15, 11, 11, 15), Sigma = 1110.36
Najden maksimum na 52 vozliščih za f(n) = 0.3: (15, 11, 11, 15), Sigma = 1170.12
Najden maksimum na 52 vozliščih za f(n) = 0.4: (16, 10, 10, 16), Sigma = 1239.51
Najden maksimum na 52 vozliščih za f(n) = 0.5: (17, 9, 9, 17), Sigma = 1320.31
Najden maksimum na 52 vozliščih za f(n) = 0.6: (19, 7, 7, 19), Sigma = 1416.06
Najden maksimum na 52 vozliščih za f(n) = 0.7: (20, 6, 6, 20), Sigma = 1528.95
Najden maksimum na 52 vozliščih za f(n) = 0.8: (21, 5, 5, 21), Sigma = 1662.66
Najden maksimum na 52 vozliščih za f(n) = 0.9: (22, 3, 4, 23), Sigma = 1823.04
Simulated annealing algoritm:
Najden maksimum na 52 vozliščih za f(n) = 1/n = 0.02: (13, 13, 

In [12]:
print("Hill climbing algoritm:")
iterations = 100000
n = 53
f = lambda n: 1/n


zmagovalec, sigma = hill_climbing(n, f, iterations)
print(f"Najden maksimum na {n} vozliščih za f(n) = 1/n = {round(1/n,2)}: {degree_configuration(zmagovalec)}, Sigma = {round(sigma,2)}")

c = 0
while c < 1:
    f = lambda n: c
    zmagovalec, sigma = hill_climbing(n, f, iterations)
    print(f"Najden maksimum na {n} vozliščih za f(n) = {c}: {degree_configuration(zmagovalec)}, Sigma = {round(sigma,2)}")
    c += 0.1
    c = round(c,2)

print("Simulated annealing algoritm:")
f = lambda n: 1/n
T_initial = 1.0
alpha = 0.99

zmagovalec, sigma = hill_climbing_sa(n, f, iterations, T_initial, alpha)
print(f"Najden maksimum na {n} vozliščih za f(n) = 1/n = {round(1/n,2)}: {degree_configuration(zmagovalec)}, Sigma = {round(sigma,2)}")

c = 0
while c < 1:
    f = lambda n: c
    zmagovalec, sigma = hill_climbing_sa(n, f, iterations, T_initial, alpha)
    print(f"Najden maksimum na {n} vozliščih za f(n) = {c}: {degree_configuration(zmagovalec)}, Sigma = {round(sigma,2)}")
    c += 0.1
    c = round(c,2)

Hill climbing algoritm:
Najden maksimum na 53 vozliščih za f(n) = 1/n = 0.02: (13, 13, 13, 14), Sigma = 1061.43
Najden maksimum na 53 vozliščih za f(n) = 0: (23, 16, 7, 7), Sigma = 1378.0
Najden maksimum na 53 vozliščih za f(n) = 0.1: (14, 13, 12, 14), Sigma = 1099.88
Najden maksimum na 53 vozliščih za f(n) = 0.2: (15, 12, 11, 15), Sigma = 1153.59
Najden maksimum na 53 vozliščih za f(n) = 0.3: (16, 11, 10, 16), Sigma = 1215.6
Najden maksimum na 53 vozliščih za f(n) = 0.4: (17, 10, 9, 17), Sigma = 1287.68
Najden maksimum na 53 vozliščih za f(n) = 0.5: (18, 9, 8, 18), Sigma = 1371.93
Najden maksimum na 53 vozliščih za f(n) = 0.6: (19, 8, 7, 19), Sigma = 1470.86
Najden maksimum na 53 vozliščih za f(n) = 0.7: (20, 6, 6, 21), Sigma = 1587.85
Najden maksimum na 53 vozliščih za f(n) = 0.8: (21, 5, 5, 22), Sigma = 1726.94
Najden maksimum na 53 vozliščih za f(n) = 0.9: (23, 3, 3, 24), Sigma = 1896.82
Simulated annealing algoritm:
Najden maksimum na 53 vozliščih za f(n) = 1/n = 0.02: (13, 13, 13

In [13]:
print("Hill climbing algoritm:")
iterations = 1000000
n = 100
f = lambda n: 1/n


zmagovalec, sigma = hill_climbing(n, f, iterations)
print(f"Najden maksimum na {n} vozliščih za f(n) = 1/n = {round(1/n,2)}: {degree_configuration(zmagovalec)}, Sigma = {round(sigma,2)}")

c = 0
while c < 1:
    f = lambda n: c
    zmagovalec, sigma = hill_climbing(n, f, iterations)
    print(f"Najden maksimum na {n} vozliščih za f(n) = {c}: {degree_configuration(zmagovalec)}, Sigma = {round(sigma,2)}")
    c += 0.1
    c = round(c,2)

print("Simulated annealing algoritm:")
f = lambda n: 1/n
T_initial = 1.0
alpha = 0.99

zmagovalec, sigma = hill_climbing_sa(n, f, iterations, T_initial, alpha)
print(f"Najden maksimum na {n} vozliščih za f(n) = 1/n = {round(1/n,2)}: {degree_configuration(zmagovalec)}, Sigma = {round(sigma,2)}")

c = 0
while c < 1:
    f = lambda n: c
    zmagovalec, sigma = hill_climbing_sa(n, f, iterations, T_initial, alpha)
    print(f"Najden maksimum na {n} vozliščih za f(n) = {c}: {degree_configuration(zmagovalec)}, Sigma = {round(sigma,2)}")
    c += 0.1
    c = round(c,2)

Hill climbing algoritm:
Najden maksimum na 100 vozliščih za f(n) = 1/n = 0.01: (25, 25, 25, 25), Sigma = 3765.6
Najden maksimum na 100 vozliščih za f(n) = 0: (47, 21, 19, 13), Sigma = 4950.0
Najden maksimum na 100 vozliščih za f(n) = 0.1: (26, 24, 24, 26), Sigma = 3916.07
Najden maksimum na 100 vozliščih za f(n) = 0.2: (28, 22, 22, 28), Sigma = 4107.85
Najden maksimum na 100 vozliščih za f(n) = 0.3: (30, 20, 20, 30), Sigma = 4328.72
Najden maksimum na 100 vozliščih za f(n) = 0.4: (31, 19, 19, 31), Sigma = 4584.7
Najden maksimum na 100 vozliščih za f(n) = 0.5: (34, 16, 16, 34), Sigma = 4884.92
Najden maksimum na 100 vozliščih za f(n) = 0.6: (36, 14, 14, 36), Sigma = 5237.25
Najden maksimum na 100 vozliščih za f(n) = 0.7: (38, 11, 12, 39), Sigma = 5653.35
Najden maksimum na 100 vozliščih za f(n) = 0.8: (42, 8, 8, 42), Sigma = 6154.13
Najden maksimum na 100 vozliščih za f(n) = 0.9: (44, 5, 6, 45), Sigma = 6754.5
Simulated annealing algoritm:
Najden maksimum na 100 vozliščih za f(n) = 1/n 

In [14]:
print("Hill climbing algoritm:")
iterations = 1000000
n = 101
f = lambda n: 1/n


zmagovalec, sigma = hill_climbing(n, f, iterations)
print(f"Najden maksimum na {n} vozliščih za f(n) = 1/n = {round(1/n,2)}: {degree_configuration(zmagovalec)}, Sigma = {round(sigma,2)}")

c = 0
while c < 1:
    f = lambda n: c
    zmagovalec, sigma = hill_climbing(n, f, iterations)
    print(f"Najden maksimum na {n} vozliščih za f(n) = {c}: {degree_configuration(zmagovalec)}, Sigma = {round(sigma,2)}")
    c += 0.1
    c = round(c,2)

print("Simulated annealing algoritm:")
f = lambda n: 1/n
T_initial = 1.0
alpha = 0.99

zmagovalec, sigma = hill_climbing_sa(n, f, iterations, T_initial, alpha)
print(f"Najden maksimum na {n} vozliščih za f(n) = 1/n = {round(1/n,2)}: {degree_configuration(zmagovalec)}, Sigma = {round(sigma,2)}")

c = 0
while c < 1:
    f = lambda n: c
    zmagovalec, sigma = hill_climbing_sa(n, f, iterations, T_initial, alpha)
    print(f"Najden maksimum na {n} vozliščih za f(n) = {c}: {degree_configuration(zmagovalec)}, Sigma = {round(sigma,2)}")
    c += 0.1
    c = round(c,2)

Hill climbing algoritm:
Najden maksimum na 101 vozliščih za f(n) = 1/n = 0.01: (25, 25, 25, 26), Sigma = 3840.89
Najden maksimum na 101 vozliščih za f(n) = 0: (45, 28, 13, 15), Sigma = 5050.0
Najden maksimum na 101 vozliščih za f(n) = 0.1: (26, 24, 24, 27), Sigma = 3994.81
Najden maksimum na 101 vozliščih za f(n) = 0.2: (28, 23, 22, 28), Sigma = 4190.01
Najden maksimum na 101 vozliščih za f(n) = 0.3: (30, 21, 20, 30), Sigma = 4415.66
Najden maksimum na 101 vozliščih za f(n) = 0.4: (32, 19, 18, 32), Sigma = 4677.39
Najden maksimum na 101 vozliščih za f(n) = 0.5: (34, 17, 16, 34), Sigma = 4983.0
Najden maksimum na 101 vozliščih za f(n) = 0.6: (36, 14, 14, 37), Sigma = 5342.06
Najden maksimum na 101 vozliščih za f(n) = 0.7: (39, 12, 11, 39), Sigma = 5768.0
Najden maksimum na 101 vozliščih za f(n) = 0.8: (42, 9, 8, 42), Sigma = 6277.25
Najden maksimum na 101 vozliščih za f(n) = 0.9: (45, 5, 5, 46), Sigma = 6892.96
Simulated annealing algoritm:
Najden maksimum na 101 vozliščih za f(n) = 1/n

In [15]:
print("Hill climbing algoritm:")
iterations = 1000000
n = 102
f = lambda n: 1/n


zmagovalec, sigma = hill_climbing(n, f, iterations)
print(f"Najden maksimum na {n} vozliščih za f(n) = 1/n = {round(1/n,2)}: {degree_configuration(zmagovalec)}, Sigma = {round(sigma,2)}")

c = 0
while c < 1:
    f = lambda n: c
    zmagovalec, sigma = hill_climbing(n, f, iterations)
    print(f"Najden maksimum na {n} vozliščih za f(n) = {c}: {degree_configuration(zmagovalec)}, Sigma = {round(sigma,2)}")
    c += 0.1
    c = round(c,2)

print("Simulated annealing algoritm:")
f = lambda n: 1/n
T_initial = 1.0
alpha = 0.99

zmagovalec, sigma = hill_climbing_sa(n, f, iterations, T_initial, alpha)
print(f"Najden maksimum na {n} vozliščih za f(n) = 1/n = {round(1/n,2)}: {degree_configuration(zmagovalec)}, Sigma = {round(sigma,2)}")

c = 0
while c < 1:
    f = lambda n: c
    zmagovalec, sigma = hill_climbing_sa(n, f, iterations, T_initial, alpha)
    print(f"Najden maksimum na {n} vozliščih za f(n) = {c}: {degree_configuration(zmagovalec)}, Sigma = {round(sigma,2)}")
    c += 0.1
    c = round(c,2)

Hill climbing algoritm:
Najden maksimum na 102 vozliščih za f(n) = 1/n = 0.01: (25, 26, 25, 26), Sigma = 3916.91
Najden maksimum na 102 vozliščih za f(n) = 0: (42, 33, 14, 13), Sigma = 5151.0
Najden maksimum na 102 vozliščih za f(n) = 0.1: (26, 25, 24, 27), Sigma = 4073.75
Najden maksimum na 102 vozliščih za f(n) = 0.2: (29, 22, 23, 28), Sigma = 4273.31
Najden maksimum na 102 vozliščih za f(n) = 0.3: (31, 20, 21, 30), Sigma = 4503.22
Najden maksimum na 102 vozliščih za f(n) = 0.4: (32, 19, 18, 33), Sigma = 4770.12
Najden maksimum na 102 vozliščih za f(n) = 0.5: (34, 17, 16, 35), Sigma = 5081.93
Najden maksimum na 102 vozliščih za f(n) = 0.6: (36, 15, 14, 37), Sigma = 5448.14
Najden maksimum na 102 vozliščih za f(n) = 0.7: (39, 11, 13, 39), Sigma = 5881.35
Najden maksimum na 102 vozliščih za f(n) = 0.8: (42, 8, 10, 42), Sigma = 6400.38
Najden maksimum na 102 vozliščih za f(n) = 0.9: (46, 4, 6, 46), Sigma = 7029.93
Simulated annealing algoritm:
Najden maksimum na 102 vozliščih za f(n) = 

In [19]:
print("Hill climbing algoritm:")
iterations = 100000
n = 103
f = lambda n: 1/n


zmagovalec, sigma = hill_climbing(n, f, iterations)
print(f"Najden maksimum na {n} vozliščih za f(n) = 1/n = {round(1/n,2)}: {degree_configuration(zmagovalec)}, Sigma = {round(sigma,2)}")

c = 0
while c < 1:
    f = lambda n: c
    zmagovalec, sigma = hill_climbing(n, f, iterations)
    print(f"Najden maksimum na {n} vozliščih za f(n) = {c}: {degree_configuration(zmagovalec)}, Sigma = {round(sigma,2)}")
    c += 0.1
    c = round(c,2)

print("Simulated annealing algoritm:")
f = lambda n: 1/n
T_initial = 1.0
alpha = 0.99

zmagovalec, sigma = hill_climbing_sa(n, f, iterations, T_initial, alpha)
print(f"Najden maksimum na {n} vozliščih za f(n) = 1/n = {round(1/n,2)}: {degree_configuration(zmagovalec)}, Sigma = {round(sigma,2)}")

c = 0
while c < 1:
    f = lambda n: c
    zmagovalec, sigma = hill_climbing_sa(n, f, iterations, T_initial, alpha)
    print(f"Najden maksimum na {n} vozliščih za f(n) = {c}: {degree_configuration(zmagovalec)}, Sigma = {round(sigma,2)}")
    c += 0.1
    c = round(c,2)

Hill climbing algoritm:
Najden maksimum na 103 vozliščih za f(n) = 1/n = 0.01: (26, 25, 26, 26), Sigma = 3994.2
Najden maksimum na 103 vozliščih za f(n) = 0: (49, 25, 11, 18), Sigma = 5253.0
Najden maksimum na 103 vozliščih za f(n) = 0.1: (27, 24, 25, 27), Sigma = 4154.61
Najden maksimum na 103 vozliščih za f(n) = 0.2: (29, 22, 23, 29), Sigma = 4357.71
Najden maksimum na 103 vozliščih za f(n) = 0.3: (31, 21, 21, 30), Sigma = 4592.16
Najden maksimum na 103 vozliščih za f(n) = 0.4: (33, 19, 19, 32), Sigma = 4864.34
Najden maksimum na 103 vozliščih za f(n) = 0.5: (35, 16, 17, 35), Sigma = 5182.18
Najden maksimum na 103 vozliščih za f(n) = 0.6: (37, 14, 15, 37), Sigma = 5555.89
Najden maksimum na 103 vozliščih za f(n) = 0.7: (40, 11, 12, 40), Sigma = 5998.82
Najden maksimum na 103 vozliščih za f(n) = 0.8: (43, 8, 9, 43), Sigma = 6528.55
Najden maksimum na 103 vozliščih za f(n) = 0.9: (47, 4, 5, 47), Sigma = 7169.86
Simulated annealing algoritm:
Najden maksimum na 103 vozliščih za f(n) = 1/

In [20]:
print("Hill climbing algoritm:")
iterations = 100000
n = 1000
f = lambda n: 1/n


zmagovalec, sigma = hill_climbing(n, f, iterations)
print(f"Najden maksimum na {n} vozliščih za f(n) = 1/n = {round(1/n,2)}: {degree_configuration(zmagovalec)}, Sigma = {round(sigma,2)}")

c = 0
while c < 1:
    f = lambda n: c
    zmagovalec, sigma = hill_climbing(n, f, iterations)
    print(f"Najden maksimum na {n} vozliščih za f(n) = {c}: {degree_configuration(zmagovalec)}, Sigma = {round(sigma,2)}")
    c += 0.1
    c = round(c,2)

print("Simulated annealing algoritm:")
f = lambda n: 1/n
T_initial = 1.0
alpha = 0.99

zmagovalec, sigma = hill_climbing_sa(n, f, iterations, T_initial, alpha)
print(f"Najden maksimum na {n} vozliščih za f(n) = 1/n = {round(1/n,2)}: {degree_configuration(zmagovalec)}, Sigma = {round(sigma,2)}")

c = 0
while c < 1:
    f = lambda n: c
    zmagovalec, sigma = hill_climbing_sa(n, f, iterations, T_initial, alpha)
    print(f"Najden maksimum na {n} vozliščih za f(n) = {c}: {degree_configuration(zmagovalec)}, Sigma = {round(sigma,2)}")
    c += 0.1
    c = round(c,2)

Hill climbing algoritm:
Najden maksimum na 1000 vozliščih za f(n) = 1/n = 0.0: (250, 250, 250, 250), Sigma = 375155.37
Najden maksimum na 1000 vozliščih za f(n) = 0: (461, 237, 145, 157), Sigma = 499500.0
Najden maksimum na 1000 vozliščih za f(n) = 0.1: (264, 236, 236, 264), Sigma = 391644.87
Najden maksimum na 1000 vozliščih za f(n) = 0.2: (280, 220, 220, 280), Sigma = 410784.94
Najden maksimum na 1000 vozliščih za f(n) = 0.3: (297, 203, 203, 297), Sigma = 432889.69
Najden maksimum na 1000 vozliščih za f(n) = 0.4: (316, 184, 184, 316), Sigma = 458548.03
Najden maksimum na 1000 vozliščih za f(n) = 0.5: (337, 163, 163, 337), Sigma = 488506.61
Najden maksimum na 1000 vozliščih za f(n) = 0.6: (346, 149, 142, 363), Sigma = 523512.55
Najden maksimum na 1000 vozliščih za f(n) = 0.7: (357, 136, 115, 392), Sigma = 564524.51
Najden maksimum na 1000 vozliščih za f(n) = 0.8: (357, 134, 85, 424), Sigma = 611552.27
Najden maksimum na 1000 vozliščih za f(n) = 0.9: (354, 128, 58, 460), Sigma = 665297

In [21]:
print("Hill climbing algoritm:")
iterations = 100000
n = 1001
f = lambda n: 1/n


zmagovalec, sigma = hill_climbing(n, f, iterations)
print(f"Najden maksimum na {n} vozliščih za f(n) = 1/n = {round(1/n,2)}: {degree_configuration(zmagovalec)}, Sigma = {round(sigma,2)}")

c = 0
while c < 1:
    f = lambda n: c
    zmagovalec, sigma = hill_climbing(n, f, iterations)
    print(f"Najden maksimum na {n} vozliščih za f(n) = {c}: {degree_configuration(zmagovalec)}, Sigma = {round(sigma,2)}")
    c += 0.1
    c = round(c,2)

print("Simulated annealing algoritm:")
f = lambda n: 1/n
T_initial = 1.0
alpha = 0.99

zmagovalec, sigma = hill_climbing_sa(n, f, iterations, T_initial, alpha)
print(f"Najden maksimum na {n} vozliščih za f(n) = 1/n = {round(1/n,2)}: {degree_configuration(zmagovalec)}, Sigma = {round(sigma,2)}")

c = 0
while c < 1:
    f = lambda n: c
    zmagovalec, sigma = hill_climbing_sa(n, f, iterations, T_initial, alpha)
    print(f"Najden maksimum na {n} vozliščih za f(n) = {c}: {degree_configuration(zmagovalec)}, Sigma = {round(sigma,2)}")
    c += 0.1
    c = round(c,2)

Hill climbing algoritm:
Najden maksimum na 1001 vozliščih za f(n) = 1/n = 0.0: (250, 250, 250, 251), Sigma = 375905.67
Najden maksimum na 1001 vozliščih za f(n) = 0: (456, 252, 132, 161), Sigma = 500500.0
Najden maksimum na 1001 vozliščih za f(n) = 0.1: (264, 236, 236, 265), Sigma = 392428.46
Najden maksimum na 1001 vozliščih za f(n) = 0.2: (280, 221, 220, 280), Sigma = 411606.58
Najden maksimum na 1001 vozliščih za f(n) = 0.3: (297, 203, 203, 298), Sigma = 433755.56
Najden maksimum na 1001 vozliščih za f(n) = 0.4: (316, 184, 184, 317), Sigma = 459465.2
Najden maksimum na 1001 vozliščih za f(n) = 0.5: (337, 163, 163, 338), Sigma = 489483.83
Najden maksimum na 1001 vozliščih za f(n) = 0.6: (361, 139, 139, 362), Sigma = 524775.06
Najden maksimum na 1001 vozliščih za f(n) = 0.7: (357, 137, 117, 390), Sigma = 565631.53
Najden maksimum na 1001 vozliščih za f(n) = 0.8: (349, 141, 87, 424), Sigma = 611678.5
Najden maksimum na 1001 vozliščih za f(n) = 0.9: (361, 129, 51, 460), Sigma = 668045.0

In [22]:
print("Hill climbing algoritm:")
iterations = 100000
n = 1002
f = lambda n: 1/n


zmagovalec, sigma = hill_climbing(n, f, iterations)
print(f"Najden maksimum na {n} vozliščih za f(n) = 1/n = {round(1/n,2)}: {degree_configuration(zmagovalec)}, Sigma = {round(sigma,2)}")

c = 0
while c < 1:
    f = lambda n: c
    zmagovalec, sigma = hill_climbing(n, f, iterations)
    print(f"Najden maksimum na {n} vozliščih za f(n) = {c}: {degree_configuration(zmagovalec)}, Sigma = {round(sigma,2)}")
    c += 0.1
    c = round(c,2)

print("Simulated annealing algoritm:")
f = lambda n: 1/n
T_initial = 1.0
alpha = 0.99

zmagovalec, sigma = hill_climbing_sa(n, f, iterations, T_initial, alpha)
print(f"Najden maksimum na {n} vozliščih za f(n) = 1/n = {round(1/n,2)}: {degree_configuration(zmagovalec)}, Sigma = {round(sigma,2)}")

c = 0
while c < 1:
    f = lambda n: c
    zmagovalec, sigma = hill_climbing_sa(n, f, iterations, T_initial, alpha)
    print(f"Najden maksimum na {n} vozliščih za f(n) = {c}: {degree_configuration(zmagovalec)}, Sigma = {round(sigma,2)}")
    c += 0.1
    c = round(c,2)

Hill climbing algoritm:
Najden maksimum na 1002 vozliščih za f(n) = 1/n = 0.0: (250, 251, 250, 251), Sigma = 376656.69
Najden maksimum na 1002 vozliščih za f(n) = 0: (451, 262, 129, 160), Sigma = 501501.0
Najden maksimum na 1002 vozliščih za f(n) = 0.1: (264, 237, 236, 265), Sigma = 393212.48
Najden maksimum na 1002 vozliščih za f(n) = 0.2: (280, 221, 220, 281), Sigma = 412429.25
Najden maksimum na 1002 vozliščih za f(n) = 0.3: (298, 203, 204, 297), Sigma = 434622.44
Najden maksimum na 1002 vozliščih za f(n) = 0.4: (317, 184, 185, 316), Sigma = 460383.48
Najden maksimum na 1002 vozliščih za f(n) = 0.5: (337, 164, 163, 338), Sigma = 490461.83
Najden maksimum na 1002 vozliščih za f(n) = 0.6: (351, 146, 141, 364), Sigma = 525712.64
Najden maksimum na 1002 vozliščih za f(n) = 0.7: (367, 129, 115, 391), Sigma = 567262.69
Najden maksimum na 1002 vozliščih za f(n) = 0.8: (354, 138, 86, 424), Sigma = 613530.28
Najden maksimum na 1002 vozliščih za f(n) = 0.9: (348, 136, 58, 460), Sigma = 666576

In [23]:
print("Hill climbing algoritm:")
iterations = 100000
n = 1003
f = lambda n: 1/n


zmagovalec, sigma = hill_climbing(n, f, iterations)
print(f"Najden maksimum na {n} vozliščih za f(n) = 1/n = {round(1/n,2)}: {degree_configuration(zmagovalec)}, Sigma = {round(sigma,2)}")

c = 0
while c < 1:
    f = lambda n: c
    zmagovalec, sigma = hill_climbing(n, f, iterations)
    print(f"Najden maksimum na {n} vozliščih za f(n) = {c}: {degree_configuration(zmagovalec)}, Sigma = {round(sigma,2)}")
    c += 0.1
    c = round(c,2)

print("Simulated annealing algoritm:")
f = lambda n: 1/n
T_initial = 1.0
alpha = 0.99

zmagovalec, sigma = hill_climbing_sa(n, f, iterations, T_initial, alpha)
print(f"Najden maksimum na {n} vozliščih za f(n) = 1/n = {round(1/n,2)}: {degree_configuration(zmagovalec)}, Sigma = {round(sigma,2)}")

c = 0
while c < 1:
    f = lambda n: c
    zmagovalec, sigma = hill_climbing_sa(n, f, iterations, T_initial, alpha)
    print(f"Najden maksimum na {n} vozliščih za f(n) = {c}: {degree_configuration(zmagovalec)}, Sigma = {round(sigma,2)}")
    c += 0.1
    c = round(c,2)

Hill climbing algoritm:
Najden maksimum na 1003 vozliščih za f(n) = 1/n = 0.0: (251, 250, 251, 251), Sigma = 377408.98
Najden maksimum na 1003 vozliščih za f(n) = 0: (467, 224, 159, 153), Sigma = 502503.0
Najden maksimum na 1003 vozliščih za f(n) = 0.1: (265, 236, 237, 265), Sigma = 393998.19
Najden maksimum na 1003 vozliščih za f(n) = 0.2: (281, 220, 221, 281), Sigma = 413253.01
Najden maksimum na 1003 vozliščih za f(n) = 0.3: (298, 203, 204, 298), Sigma = 435490.7
Najden maksimum na 1003 vozliščih za f(n) = 0.4: (317, 184, 185, 317), Sigma = 461303.21
Najden maksimum na 1003 vozliščih za f(n) = 0.5: (338, 163, 164, 338), Sigma = 491441.78
Najden maksimum na 1003 vozliščih za f(n) = 0.6: (356, 144, 140, 363), Sigma = 526838.1
Najden maksimum na 1003 vozliščih za f(n) = 0.7: (349, 144, 117, 393), Sigma = 567291.82
Najden maksimum na 1003 vozliščih za f(n) = 0.8: (375, 119, 85, 424), Sigma = 617034.32
Najden maksimum na 1003 vozliščih za f(n) = 0.9: (346, 147, 50, 460), Sigma = 667480.5

In [24]:
print("Hill climbing algoritm:")
iterations = 100000
n = 15
f = lambda n: c
c = 0.001

zmagovalec, sigma = hill_climbing(n, f, iterations)
print(f"Najden maksimum na {n} vozliščih za f(n) = {c}: {degree_configuration(zmagovalec)}, Sigma = {round(sigma,2)}")

c = 0.995 

zmagovalec, sigma = hill_climbing(n, f, iterations)
print(f"Najden maksimum na {n} vozliščih za f(n) = {c}: {degree_configuration(zmagovalec)}, Sigma = {round(sigma,2)}")

print("Simulated annealing algoritm:")
f = lambda n: c
T_initial = 1.0
alpha = 0.99

c = 0.001
zmagovalec, sigma = hill_climbing_sa(n, f, iterations, T_initial, alpha)
print(f"Najden maksimum na {n} vozliščih za f(n) = {c}: {degree_configuration(zmagovalec)}, Sigma = {round(sigma,2)}")

c = 0.995
zmagovalec, sigma = hill_climbing_sa(n, f, iterations, T_initial, alpha)
print(f"Najden maksimum na {n} vozliščih za f(n) = {c}: {degree_configuration(zmagovalec)}, Sigma = {round(sigma,2)}")

Hill climbing algoritm:
Najden maksimum na 15 vozliščih za f(n) = 0.001: (4, 3, 4, 4), Sigma = 84.04
Najden maksimum na 15 vozliščih za f(n) = 0.995: (7, 0, 1, 7), Sigma = 167.15
Simulated annealing algoritm:
Najden maksimum na 15 vozliščih za f(n) = 0.001: (4, 3, 4, 4), Sigma = 84.04
Najden maksimum na 15 vozliščih za f(n) = 0.995: (7, 0, 1, 7), Sigma = 167.15


In [25]:
print("Hill climbing algoritm:")
iterations = 100000
n = 16
f = lambda n: c
c = 0.001

zmagovalec, sigma = hill_climbing(n, f, iterations)
print(f"Najden maksimum na {n} vozliščih za f(n) = {c}: {degree_configuration(zmagovalec)}, Sigma = {round(sigma,2)}")

c = 0.995 

zmagovalec, sigma = hill_climbing(n, f, iterations)
print(f"Najden maksimum na {n} vozliščih za f(n) = {c}: {degree_configuration(zmagovalec)}, Sigma = {round(sigma,2)}")

print("Simulated annealing algoritm:")
f = lambda n: c
T_initial = 1.0
alpha = 0.99

c = 0.001
zmagovalec, sigma = hill_climbing_sa(n, f, iterations, T_initial, alpha)
print(f"Najden maksimum na {n} vozliščih za f(n) = {c}: {degree_configuration(zmagovalec)}, Sigma = {round(sigma,2)}")

c = 0.995
zmagovalec, sigma = hill_climbing_sa(n, f, iterations, T_initial, alpha)
print(f"Najden maksimum na {n} vozliščih za f(n) = {c}: {degree_configuration(zmagovalec)}, Sigma = {round(sigma,2)}")

Hill climbing algoritm:
Najden maksimum na 16 vozliščih za f(n) = 0.001: (4, 4, 4, 4), Sigma = 96.04
Najden maksimum na 16 vozliščih za f(n) = 0.995: (6, 1, 2, 7), Sigma = 185.18
Simulated annealing algoritm:
Najden maksimum na 16 vozliščih za f(n) = 0.001: (4, 4, 4, 4), Sigma = 96.04
Najden maksimum na 16 vozliščih za f(n) = 0.995: (6, 1, 2, 7), Sigma = 185.18


In [26]:
print("Hill climbing algoritm:")
iterations = 100000
n = 17
f = lambda n: c
c = 0.001

zmagovalec, sigma = hill_climbing(n, f, iterations)
print(f"Najden maksimum na {n} vozliščih za f(n) = {c}: {degree_configuration(zmagovalec)}, Sigma = {round(sigma,2)}")

c = 0.995 

zmagovalec, sigma = hill_climbing(n, f, iterations)
print(f"Najden maksimum na {n} vozliščih za f(n) = {c}: {degree_configuration(zmagovalec)}, Sigma = {round(sigma,2)}")

print("Simulated annealing algoritm:")
f = lambda n: c
T_initial = 1.0
alpha = 0.99

c = 0.001
zmagovalec, sigma = hill_climbing_sa(n, f, iterations, T_initial, alpha)
print(f"Najden maksimum na {n} vozliščih za f(n) = {c}: {degree_configuration(zmagovalec)}, Sigma = {round(sigma,2)}")

c = 0.995
zmagovalec, sigma = hill_climbing_sa(n, f, iterations, T_initial, alpha)
print(f"Najden maksimum na {n} vozliščih za f(n) = {c}: {degree_configuration(zmagovalec)}, Sigma = {round(sigma,2)}")

Hill climbing algoritm:
Najden maksimum na 17 vozliščih za f(n) = 0.001: (4, 4, 4, 5), Sigma = 108.05
Najden maksimum na 17 vozliščih za f(n) = 0.995: (7, 1, 1, 8), Sigma = 212.98
Simulated annealing algoritm:
Najden maksimum na 17 vozliščih za f(n) = 0.001: (4, 4, 4, 5), Sigma = 108.05
Najden maksimum na 17 vozliščih za f(n) = 0.995: (7, 1, 1, 8), Sigma = 212.98


In [27]:
print("Hill climbing algoritm:")
iterations = 100000
n = 18
f = lambda n: c
c = 0.001

zmagovalec, sigma = hill_climbing(n, f, iterations)
print(f"Najden maksimum na {n} vozliščih za f(n) = {c}: {degree_configuration(zmagovalec)}, Sigma = {round(sigma,2)}")

c = 0.995 

zmagovalec, sigma = hill_climbing(n, f, iterations)
print(f"Najden maksimum na {n} vozliščih za f(n) = {c}: {degree_configuration(zmagovalec)}, Sigma = {round(sigma,2)}")

print("Simulated annealing algoritm:")
f = lambda n: c
T_initial = 1.0
alpha = 0.99

c = 0.001
zmagovalec, sigma = hill_climbing_sa(n, f, iterations, T_initial, alpha)
print(f"Najden maksimum na {n} vozliščih za f(n) = {c}: {degree_configuration(zmagovalec)}, Sigma = {round(sigma,2)}")

c = 0.995
zmagovalec, sigma = hill_climbing_sa(n, f, iterations, T_initial, alpha)
print(f"Najden maksimum na {n} vozliščih za f(n) = {c}: {degree_configuration(zmagovalec)}, Sigma = {round(sigma,2)}")

Hill climbing algoritm:
Najden maksimum na 18 vozliščih za f(n) = 0.001: (5, 4, 5, 4), Sigma = 121.05
Najden maksimum na 18 vozliščih za f(n) = 0.995: (8, 0, 2, 8), Sigma = 238.84
Simulated annealing algoritm:
Najden maksimum na 18 vozliščih za f(n) = 0.001: (5, 4, 5, 4), Sigma = 121.05
Najden maksimum na 18 vozliščih za f(n) = 0.995: (8, 1, 0, 9), Sigma = 240.75


In [28]:
print("Hill climbing algoritm:")
iterations = 100000
n = 19
f = lambda n: c
c = 0.001

zmagovalec, sigma = hill_climbing(n, f, iterations)
print(f"Najden maksimum na {n} vozliščih za f(n) = {c}: {degree_configuration(zmagovalec)}, Sigma = {round(sigma,2)}")

c = 0.995 

zmagovalec, sigma = hill_climbing(n, f, iterations)
print(f"Najden maksimum na {n} vozliščih za f(n) = {c}: {degree_configuration(zmagovalec)}, Sigma = {round(sigma,2)}")

print("Simulated annealing algoritm:")
f = lambda n: c
T_initial = 1.0
alpha = 0.99

c = 0.001
zmagovalec, sigma = hill_climbing_sa(n, f, iterations, T_initial, alpha)
print(f"Najden maksimum na {n} vozliščih za f(n) = {c}: {degree_configuration(zmagovalec)}, Sigma = {round(sigma,2)}")

c = 0.995
zmagovalec, sigma = hill_climbing_sa(n, f, iterations, T_initial, alpha)
print(f"Najden maksimum na {n} vozliščih za f(n) = {c}: {degree_configuration(zmagovalec)}, Sigma = {round(sigma,2)}")

Hill climbing algoritm:
Najden maksimum na 19 vozliščih za f(n) = 0.001: (5, 4, 5, 5), Sigma = 135.06
Najden maksimum na 19 vozliščih za f(n) = 0.995: (9, 0, 1, 9), Sigma = 268.61
Simulated annealing algoritm:
Najden maksimum na 19 vozliščih za f(n) = 0.001: (5, 4, 5, 5), Sigma = 135.06
Najden maksimum na 19 vozliščih za f(n) = 0.995: (8, 1, 0, 10), Sigma = 266.62


In [29]:
print("Hill climbing algoritm:")
iterations = 100000
n = 20
f = lambda n: c
c = 0.001

zmagovalec, sigma = hill_climbing(n, f, iterations)
print(f"Najden maksimum na {n} vozliščih za f(n) = {c}: {degree_configuration(zmagovalec)}, Sigma = {round(sigma,2)}")

c = 0.995 

zmagovalec, sigma = hill_climbing(n, f, iterations)
print(f"Najden maksimum na {n} vozliščih za f(n) = {c}: {degree_configuration(zmagovalec)}, Sigma = {round(sigma,2)}")

print("Simulated annealing algoritm:")
f = lambda n: c
T_initial = 1.0
alpha = 0.99

c = 0.001
zmagovalec, sigma = hill_climbing_sa(n, f, iterations, T_initial, alpha)
print(f"Najden maksimum na {n} vozliščih za f(n) = {c}: {degree_configuration(zmagovalec)}, Sigma = {round(sigma,2)}")

c = 0.995
zmagovalec, sigma = hill_climbing_sa(n, f, iterations, T_initial, alpha)
print(f"Najden maksimum na {n} vozliščih za f(n) = {c}: {degree_configuration(zmagovalec)}, Sigma = {round(sigma,2)}")

Hill climbing algoritm:
Najden maksimum na 20 vozliščih za f(n) = 0.001: (5, 5, 5, 5), Sigma = 150.06
Najden maksimum na 20 vozliščih za f(n) = 0.995: (8, 1, 2, 9), Sigma = 292.64
Simulated annealing algoritm:
Najden maksimum na 20 vozliščih za f(n) = 0.001: (5, 5, 5, 5), Sigma = 150.06
Najden maksimum na 20 vozliščih za f(n) = 0.995: (8, 1, 2, 9), Sigma = 292.64


In [30]:
print("Hill climbing algoritm:")
iterations = 100000
n = 50
f = lambda n: c
c = 0.001

zmagovalec, sigma = hill_climbing(n, f, iterations)
print(f"Najden maksimum na {n} vozliščih za f(n) = {c}: {degree_configuration(zmagovalec)}, Sigma = {round(sigma,2)}")

c = 0.995 

zmagovalec, sigma = hill_climbing(n, f, iterations)
print(f"Najden maksimum na {n} vozliščih za f(n) = {c}: {degree_configuration(zmagovalec)}, Sigma = {round(sigma,2)}")

print("Simulated annealing algoritm:")
f = lambda n: c
T_initial = 1.0
alpha = 0.99

c = 0.001
zmagovalec, sigma = hill_climbing_sa(n, f, iterations, T_initial, alpha)
print(f"Najden maksimum na {n} vozliščih za f(n) = {c}: {degree_configuration(zmagovalec)}, Sigma = {round(sigma,2)}")

c = 0.995
zmagovalec, sigma = hill_climbing_sa(n, f, iterations, T_initial, alpha)
print(f"Najden maksimum na {n} vozliščih za f(n) = {c}: {degree_configuration(zmagovalec)}, Sigma = {round(sigma,2)}")

Hill climbing algoritm:
Najden maksimum na 50 vozliščih za f(n) = 0.001: (12, 13, 12, 13), Sigma = 937.39
Najden maksimum na 50 vozliščih za f(n) = 0.995: (21, 4, 1, 24), Sigma = 1848.91
Simulated annealing algoritm:
Najden maksimum na 50 vozliščih za f(n) = 0.001: (12, 13, 12, 13), Sigma = 937.39
Najden maksimum na 50 vozliščih za f(n) = 0.995: (24, 0, 2, 24), Sigma = 1862.2


In [31]:
print("Hill climbing algoritm:")
iterations = 100000
n = 51
f = lambda n: c
c = 0.001

zmagovalec, sigma = hill_climbing(n, f, iterations)
print(f"Najden maksimum na {n} vozliščih za f(n) = {c}: {degree_configuration(zmagovalec)}, Sigma = {round(sigma,2)}")

c = 0.995 

zmagovalec, sigma = hill_climbing(n, f, iterations)
print(f"Najden maksimum na {n} vozliščih za f(n) = {c}: {degree_configuration(zmagovalec)}, Sigma = {round(sigma,2)}")

print("Simulated annealing algoritm:")
f = lambda n: c
T_initial = 1.0
alpha = 0.99

c = 0.001
zmagovalec, sigma = hill_climbing_sa(n, f, iterations, T_initial, alpha)
print(f"Najden maksimum na {n} vozliščih za f(n) = {c}: {degree_configuration(zmagovalec)}, Sigma = {round(sigma,2)}")

c = 0.995
zmagovalec, sigma = hill_climbing_sa(n, f, iterations, T_initial, alpha)
print(f"Najden maksimum na {n} vozliščih za f(n) = {c}: {degree_configuration(zmagovalec)}, Sigma = {round(sigma,2)}")

Hill climbing algoritm:
Najden maksimum na 51 vozliščih za f(n) = 0.001: (13, 12, 13, 13), Sigma = 975.41
Najden maksimum na 51 vozliščih za f(n) = 0.995: (24, 1, 0, 26), Sigma = 1937.57
Simulated annealing algoritm:
Najden maksimum na 51 vozliščih za f(n) = 0.001: (13, 12, 13, 13), Sigma = 975.41
Najden maksimum na 51 vozliščih za f(n) = 0.995: (23, 1, 1, 26), Sigma = 1931.83


In [32]:
print("Hill climbing algoritm:")
iterations = 100000
n = 52
f = lambda n: c
c = 0.001

zmagovalec, sigma = hill_climbing(n, f, iterations)
print(f"Najden maksimum na {n} vozliščih za f(n) = {c}: {degree_configuration(zmagovalec)}, Sigma = {round(sigma,2)}")

c = 0.995 

zmagovalec, sigma = hill_climbing(n, f, iterations)
print(f"Najden maksimum na {n} vozliščih za f(n) = {c}: {degree_configuration(zmagovalec)}, Sigma = {round(sigma,2)}")

print("Simulated annealing algoritm:")
f = lambda n: c
T_initial = 1.0
alpha = 0.99

c = 0.001
zmagovalec, sigma = hill_climbing_sa(n, f, iterations, T_initial, alpha)
print(f"Najden maksimum na {n} vozliščih za f(n) = {c}: {degree_configuration(zmagovalec)}, Sigma = {round(sigma,2)}")

c = 0.995
zmagovalec, sigma = hill_climbing_sa(n, f, iterations, T_initial, alpha)
print(f"Najden maksimum na {n} vozliščih za f(n) = {c}: {degree_configuration(zmagovalec)}, Sigma = {round(sigma,2)}")

Hill climbing algoritm:
Najden maksimum na 52 vozliščih za f(n) = 0.001: (13, 13, 13, 13), Sigma = 1014.42
Najden maksimum na 52 vozliščih za f(n) = 0.995: (22, 3, 2, 25), Sigma = 2000.14
Simulated annealing algoritm:
Najden maksimum na 52 vozliščih za f(n) = 0.001: (13, 13, 13, 13), Sigma = 1014.42
Najden maksimum na 52 vozliščih za f(n) = 0.995: (24, 1, 2, 25), Sigma = 2011.63


In [33]:
print("Hill climbing algoritm:")
iterations = 100000
n = 53
f = lambda n: c
c = 0.001

zmagovalec, sigma = hill_climbing(n, f, iterations)
print(f"Najden maksimum na {n} vozliščih za f(n) = {c}: {degree_configuration(zmagovalec)}, Sigma = {round(sigma,2)}")

c = 0.995 

zmagovalec, sigma = hill_climbing(n, f, iterations)
print(f"Najden maksimum na {n} vozliščih za f(n) = {c}: {degree_configuration(zmagovalec)}, Sigma = {round(sigma,2)}")

print("Simulated annealing algoritm:")
f = lambda n: c
T_initial = 1.0
alpha = 0.99

c = 0.001
zmagovalec, sigma = hill_climbing_sa(n, f, iterations, T_initial, alpha)
print(f"Najden maksimum na {n} vozliščih za f(n) = {c}: {degree_configuration(zmagovalec)}, Sigma = {round(sigma,2)}")

c = 0.995
zmagovalec, sigma = hill_climbing_sa(n, f, iterations, T_initial, alpha)
print(f"Najden maksimum na {n} vozliščih za f(n) = {c}: {degree_configuration(zmagovalec)}, Sigma = {round(sigma,2)}")

Hill climbing algoritm:
Najden maksimum na 53 vozliščih za f(n) = 0.001: (13, 13, 13, 14), Sigma = 1053.44
Najden maksimum na 53 vozliščih za f(n) = 0.995: (23, 3, 1, 26), Sigma = 2083.47
Simulated annealing algoritm:
Najden maksimum na 53 vozliščih za f(n) = 0.001: (13, 13, 13, 14), Sigma = 1053.44
Najden maksimum na 53 vozliščih za f(n) = 0.995: (25, 1, 1, 26), Sigma = 2092.97


In [34]:
print("Hill climbing algoritm:")
iterations = 100000
n = 100
f = lambda n: c
c = 0.001

zmagovalec, sigma = hill_climbing(n, f, iterations)
print(f"Najden maksimum na {n} vozliščih za f(n) = {c}: {degree_configuration(zmagovalec)}, Sigma = {round(sigma,2)}")

c = 0.995 

zmagovalec, sigma = hill_climbing(n, f, iterations)
print(f"Najden maksimum na {n} vozliščih za f(n) = {c}: {degree_configuration(zmagovalec)}, Sigma = {round(sigma,2)}")

print("Simulated annealing algoritm:")
f = lambda n: c
T_initial = 1.0
alpha = 0.99

c = 0.001
zmagovalec, sigma = hill_climbing_sa(n, f, iterations, T_initial, alpha)
print(f"Najden maksimum na {n} vozliščih za f(n) = {c}: {degree_configuration(zmagovalec)}, Sigma = {round(sigma,2)}")

c = 0.995
zmagovalec, sigma = hill_climbing_sa(n, f, iterations, T_initial, alpha)
print(f"Najden maksimum na {n} vozliščih za f(n) = {c}: {degree_configuration(zmagovalec)}, Sigma = {round(sigma,2)}")

Hill climbing algoritm:
Najden maksimum na 100 vozliščih za f(n) = 0.001: (25, 25, 25, 25), Sigma = 3751.55
Najden maksimum na 100 vozliščih za f(n) = 0.995: (47, 3, 1, 49), Sigma = 7450.81
Simulated annealing algoritm:
Najden maksimum na 100 vozliščih za f(n) = 0.001: (25, 25, 25, 25), Sigma = 3751.55
Najden maksimum na 100 vozliščih za f(n) = 0.995: (46, 3, 2, 49), Sigma = 7443.3


In [35]:
print("Hill climbing algoritm:")
iterations = 100000
n = 101
f = lambda n: c
c = 0.001

zmagovalec, sigma = hill_climbing(n, f, iterations)
print(f"Najden maksimum na {n} vozliščih za f(n) = {c}: {degree_configuration(zmagovalec)}, Sigma = {round(sigma,2)}")

c = 0.995 

zmagovalec, sigma = hill_climbing(n, f, iterations)
print(f"Najden maksimum na {n} vozliščih za f(n) = {c}: {degree_configuration(zmagovalec)}, Sigma = {round(sigma,2)}")

print("Simulated annealing algoritm:")
f = lambda n: c
T_initial = 1.0
alpha = 0.99

c = 0.001
zmagovalec, sigma = hill_climbing_sa(n, f, iterations, T_initial, alpha)
print(f"Najden maksimum na {n} vozliščih za f(n) = {c}: {degree_configuration(zmagovalec)}, Sigma = {round(sigma,2)}")

c = 0.995
zmagovalec, sigma = hill_climbing_sa(n, f, iterations, T_initial, alpha)
print(f"Najden maksimum na {n} vozliščih za f(n) = {c}: {degree_configuration(zmagovalec)}, Sigma = {round(sigma,2)}")

Hill climbing algoritm:
Najden maksimum na 101 vozliščih za f(n) = 0.001: (25, 25, 25, 26), Sigma = 3826.6
Najden maksimum na 101 vozliščih za f(n) = 0.995: (45, 5, 1, 50), Sigma = 7580.98
Simulated annealing algoritm:
Najden maksimum na 101 vozliščih za f(n) = 0.001: (25, 25, 25, 26), Sigma = 3826.6
Najden maksimum na 101 vozliščih za f(n) = 0.995: (44, 6, 0, 51), Sigma = 7569.0


In [36]:
print("Hill climbing algoritm:")
iterations = 100000
n = 102
f = lambda n: c
c = 0.001

zmagovalec, sigma = hill_climbing(n, f, iterations)
print(f"Najden maksimum na {n} vozliščih za f(n) = {c}: {degree_configuration(zmagovalec)}, Sigma = {round(sigma,2)}")

c = 0.995 

zmagovalec, sigma = hill_climbing(n, f, iterations)
print(f"Najden maksimum na {n} vozliščih za f(n) = {c}: {degree_configuration(zmagovalec)}, Sigma = {round(sigma,2)}")

print("Simulated annealing algoritm:")
f = lambda n: c
T_initial = 1.0
alpha = 0.99

c = 0.001
zmagovalec, sigma = hill_climbing_sa(n, f, iterations, T_initial, alpha)
print(f"Najden maksimum na {n} vozliščih za f(n) = {c}: {degree_configuration(zmagovalec)}, Sigma = {round(sigma,2)}")

c = 0.995
zmagovalec, sigma = hill_climbing_sa(n, f, iterations, T_initial, alpha)
print(f"Najden maksimum na {n} vozliščih za f(n) = {c}: {degree_configuration(zmagovalec)}, Sigma = {round(sigma,2)}")

Hill climbing algoritm:
Najden maksimum na 102 vozliščih za f(n) = 0.001: (25, 26, 25, 26), Sigma = 3902.62
Najden maksimum na 102 vozliščih za f(n) = 0.995: (44, 6, 2, 50), Sigma = 7713.16
Simulated annealing algoritm:
Najden maksimum na 102 vozliščih za f(n) = 0.001: (25, 26, 25, 26), Sigma = 3902.62
Najden maksimum na 102 vozliščih za f(n) = 0.995: (46, 4, 2, 50), Sigma = 7736.18


In [37]:
print("Hill climbing algoritm:")
iterations = 100000
n = 103
f = lambda n: c
c = 0.001

zmagovalec, sigma = hill_climbing(n, f, iterations)
print(f"Najden maksimum na {n} vozliščih za f(n) = {c}: {degree_configuration(zmagovalec)}, Sigma = {round(sigma,2)}")

c = 0.995 

zmagovalec, sigma = hill_climbing(n, f, iterations)
print(f"Najden maksimum na {n} vozliščih za f(n) = {c}: {degree_configuration(zmagovalec)}, Sigma = {round(sigma,2)}")

print("Simulated annealing algoritm:")
f = lambda n: c
T_initial = 1.0
alpha = 0.99

c = 0.001
zmagovalec, sigma = hill_climbing_sa(n, f, iterations, T_initial, alpha)
print(f"Najden maksimum na {n} vozliščih za f(n) = {c}: {degree_configuration(zmagovalec)}, Sigma = {round(sigma,2)}")

c = 0.995
zmagovalec, sigma = hill_climbing_sa(n, f, iterations, T_initial, alpha)
print(f"Najden maksimum na {n} vozliščih za f(n) = {c}: {degree_configuration(zmagovalec)}, Sigma = {round(sigma,2)}")

Hill climbing algoritm:
Najden maksimum na 103 vozliščih za f(n) = 0.001: (26, 25, 26, 26), Sigma = 3979.66
Najden maksimum na 103 vozliščih za f(n) = 0.995: (49, 2, 1, 51), Sigma = 7907.89
Simulated annealing algoritm:
Najden maksimum na 103 vozliščih za f(n) = 0.001: (26, 25, 26, 26), Sigma = 3979.66
Najden maksimum na 103 vozliščih za f(n) = 0.995: (42, 9, 0, 52), Sigma = 7826.87


In [38]:
print("Hill climbing algoritm:")
iterations = 100000
n = 1000
f = lambda n: c
c = 0.001

zmagovalec, sigma = hill_climbing(n, f, iterations)
print(f"Najden maksimum na {n} vozliščih za f(n) = {c}: {degree_configuration(zmagovalec)}, Sigma = {round(sigma,2)}")

c = 0.995 

zmagovalec, sigma = hill_climbing(n, f, iterations)
print(f"Najden maksimum na {n} vozliščih za f(n) = {c}: {degree_configuration(zmagovalec)}, Sigma = {round(sigma,2)}")

print("Simulated annealing algoritm:")
f = lambda n: c
T_initial = 1.0
alpha = 0.99

c = 0.001
zmagovalec, sigma = hill_climbing_sa(n, f, iterations, T_initial, alpha)
print(f"Najden maksimum na {n} vozliščih za f(n) = {c}: {degree_configuration(zmagovalec)}, Sigma = {round(sigma,2)}")

c = 0.995
zmagovalec, sigma = hill_climbing_sa(n, f, iterations, T_initial, alpha)
print(f"Najden maksimum na {n} vozliščih za f(n) = {c}: {degree_configuration(zmagovalec)}, Sigma = {round(sigma,2)}")

Hill climbing algoritm:
Najden maksimum na 1000 vozliščih za f(n) = 0.001: (250, 250, 250, 250), Sigma = 375155.37
Najden maksimum na 1000 vozliščih za f(n) = 0.995: (353, 112, 37, 498), Sigma = 723796.68
Simulated annealing algoritm:
Najden maksimum na 1000 vozliščih za f(n) = 0.001: (250, 250, 250, 250), Sigma = 375155.37
Najden maksimum na 1000 vozliščih za f(n) = 0.995: (351, 111, 41, 497), Sigma = 722997.83


In [39]:
print("Hill climbing algoritm:")
iterations = 100000
n = 1001
f = lambda n: c
c = 0.001

zmagovalec, sigma = hill_climbing(n, f, iterations)
print(f"Najden maksimum na {n} vozliščih za f(n) = {c}: {degree_configuration(zmagovalec)}, Sigma = {round(sigma,2)}")

c = 0.995 

zmagovalec, sigma = hill_climbing(n, f, iterations)
print(f"Najden maksimum na {n} vozliščih za f(n) = {c}: {degree_configuration(zmagovalec)}, Sigma = {round(sigma,2)}")

print("Simulated annealing algoritm:")
f = lambda n: c
T_initial = 1.0
alpha = 0.99

c = 0.001
zmagovalec, sigma = hill_climbing_sa(n, f, iterations, T_initial, alpha)
print(f"Najden maksimum na {n} vozliščih za f(n) = {c}: {degree_configuration(zmagovalec)}, Sigma = {round(sigma,2)}")

c = 0.995
zmagovalec, sigma = hill_climbing_sa(n, f, iterations, T_initial, alpha)
print(f"Najden maksimum na {n} vozliščih za f(n) = {c}: {degree_configuration(zmagovalec)}, Sigma = {round(sigma,2)}")

Hill climbing algoritm:
Najden maksimum na 1001 vozliščih za f(n) = 0.001: (250, 250, 250, 251), Sigma = 375905.82
Najden maksimum na 1001 vozliščih za f(n) = 0.995: (361, 112, 31, 497), Sigma = 727860.86
Simulated annealing algoritm:
Najden maksimum na 1001 vozliščih za f(n) = 0.001: (250, 250, 250, 251), Sigma = 375905.82
Najden maksimum na 1001 vozliščih za f(n) = 0.995: (356, 112, 34, 499), Sigma = 726172.13
